In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from database_io.db_handler import DB_handler
import pymc as pm
import numpy as np
import pandas as pd

import plotly.express as px

In [ ]:
db = DB_handler()

In [ ]:
games = db.games.get_all_games(0.1)

In [ ]:
# games["minutes_missed"] = (90 - games["minutes"]).clip(lower=0)
games["elo_diff"] = games["elo"] - games["opposition_elo"]
games["goal_diff"] = games["result"].apply(lambda x: int(x.split("-")[0]) - int(x.split("-")[1]))

In [ ]:
games["elo_diff"] = games["elo_diff"] / (elo_std:=max(games["elo_diff"].max(), abs(games["elo_diff"].min())))
# games["goal_diff"] = games["goal_diff"] / (goal_std:=max(games["goal_diff"].max(), abs(games["goal_diff"].min())))
games["minutes_missed"] = (games["game_minutes"] - games["minutes"]).clip(lower=0)

# games["minutes_missed"] = games["minutes_missed"] / (min_std:=games["minutes_missed"].max())

In [ ]:
games["minutes_missed"] = (games["minutes_missed"] / (minutes_missed_std:=max(games["minutes_missed"].max(), abs(games["minutes_missed"].min()))))


In [ ]:
games["minutes_missed_abs"] = games["minutes_missed"].abs()
games["goal_diff_abs"] = games["goal_diff"].abs()

In [ ]:
px.scatter(games, "goal_diff_abs", "minutes_missed_abs")

In [ ]:
print(elo_std, minutes_missed_std)

In [ ]:
games

In [ ]:
with pm.Model() as MoV:
    # mutable data
    is_home = games["home"].values
    elo_diff = games["elo_diff"].values
    minutes_missed = games["minutes_missed"].values
    goal_diff = games["goal_diff"].values

    # pymc vars
    home_advantage = pm.Normal("home_advantage", mu = 0.3, sigma = 1)
    elo_diff_coeff = pm.Normal("elo_diff_coeff", mu = 1, sigma = 1)
    minutes_missed_coeff = pm.Normal("minutes_missed_coeff", mu = 1, sigma = 1)
    mean_ = pm.Deterministic("mean_", ((is_home * home_advantage) + (elo_diff * elo_diff_coeff)) * pm.math.abs(1 - (minutes_missed * minutes_missed_coeff))) # instead of abs -> if < 0 then 0 ?
    var_ = pm.HalfNormal("var_")
    goal_diff = pm.Normal("goal_diff", mean_, var_, observed=goal_diff)

In [ ]:
with MoV:
    trace = pm.sample(10000)

In [ ]:
# (is_home * home_advantage) + (elo_diff * elo_diff_coeff) - (minutes_missed * minutes_missed_coeff))
home_test = 0
elo_diff_test = 70 / elo_std
minutes_missed_test = 45 / minutes_missed_std

(home_test * trace.posterior.home_advantage.values[0][0] + elo_diff_test * trace.posterior.elo_diff_coeff.values[0][0]) * abs(1 -  minutes_missed_test * trace.posterior.minutes_missed_coeff.values[0][0])

In [ ]:
trace.to_netcdf("/home/morten/Develop/packing-report/metrics/mov_elo/mov_elo.nc")